<a id='101-bullet'></a>
# Аналитика для книжного сервиса

Компания купила крупный сервис для чтения книг по подписке. Наша задача — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта. 
_________________________________________________________________

# 1. Подготовка данных
* [1.1 Создание подключения у базе](#1-bullet)
* [1.2 Изучение данных](#2-bullet)   

# 2. Ответы на вопросы
* [2.1 Сколько книг вышло после 1 января 2000 года?](#3-bullet)
* [2.2 Сколько обзоров вышло на книгу и какая средняя оценка для каждой книги?](#4-bullet)
* [2.3 Какое издательство выпустило наибольшее количество книг, толще 50 страниц?](#5-bullet)
* [2.4 Какой автор имеет самую высокую среднюю оценку среди книг толще 50 страниц?](#6-bullet)
* [2.5 Сколько в среднем обзоров было от пользователей, которые поставили больше 50 оценок?](#7-bullet)

__________________________________________________________________
### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
#загрузим библиотеки
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

## Подготовка данных
<a id='1-bullet'></a>
### Создание подключения у базе
[К оглавлению](#101-bullet)

In [2]:
#устанавливаем параметры
try:
    db_config = {'user': 'praktikum_student', #имя пользователя
                 'pwd': 'Sdf4$2;d-d30pp', #пароль
                 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
                 'port': 6432, #порт подключения
                 'db': 'data-analyst-final-project-db'} #название базы данных

    connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                             db_config['pwd'],
                                                             db_config['host'],
                                                             db_config['port'],
                                                             db_config['db']) 

    #сохраняем коннектор
    engine = create_engine(connection_string, connect_args={'sslmode':'require'})
    
    print('Подключение создано')
except:
      print('Не удалось создать подключение')

Подключение создано


<a id='2-bullet'></a>
### Изучение данных
[К оглавлению](#101-bullet)

In [3]:
#выведем первые строки таблиц на экран
df_name = ['books', 'ratings', 'authors', 'reviews', 'publishers']
for name in df_name:
    query = '''
    select * from {} limit 3
    '''.format(name)
    print('Таблица {}'.format(name))
    df = pd.io.sql.read_sql(query, con = engine)
    display(df)

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


Таблица publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


## Ответы на вопросы
<a id='3-bullet'></a>
### Сколько книг вышло после 1 января 2000 года?
[К оглавлению](#101-bullet)

In [4]:
#
query = '''
select
count(distinct book_id) as book_cnt
from 
books
where publication_date > '2000-01-01'
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,book_cnt
0,819


За период после 1 января 2000 года вышло 819 книг.

<a id='4-bullet'></a>
### Сколько обзоров вышло на книгу и какая средняя оценка для каждой книги?
[К оглавлению](#101-bullet)

In [5]:
#
query = '''
select 
    books.book_id,
    books.title,
    rat.avg_rating,
    cnt_review

from
    books
        left join (select 
                    book_id,
                    avg(rating) as avg_rating

                    from
                    ratings

                    group by
                    book_id
                    ) as rat on rat.book_id = books.book_id
        left join (select 
                    book_id,
                    count(distinct review_id) as cnt_review

                    from
                    reviews

                    group by
                    book_id) as rew on rew.book_id = books.book_id
order by rat.avg_rating desc
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,book_id,title,avg_rating,cnt_review
0,55,A Woman of Substance (Emma Harte Saga #1),5.00,2.0
1,182,Dead Souls,5.00,2.0
2,169,Crucial Conversations: Tools for Talking When ...,5.00,2.0
3,610,Tai-Pan (Asian Saga #2),5.00,2.0
4,972,Wherever You Go There You Are: Mindfulness Me...,5.00,2.0
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,202,Drowning Ruth,2.00,3.0
997,316,His Excellency: George Washington,2.00,2.0
998,371,Junky,2.00,2.0


В таблице выше можно найти данные по количеству обзоров и средним оценкам книги. В среднем по всей выборке книги имеет рейтинг около 3.89 и около ~2.8 обзора.

<a id='5-bullet'></a>
### Какое издательство выпустило наибольшее количество книг, толще 50 страниц?
[К оглавлению](#101-bullet)

In [6]:
#
query = '''
select
    pub.publisher,
    count(distinct title) as cnt_book

from books
    left join publishers as pub on pub.publisher_id = books.publisher_id

where num_pages > 50

group by 
    books.publisher_id,
    pub.publisher

order by
    cnt_book desc

limit 3
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,publisher,cnt_book
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


Больше всего книг толще 50 страниц выпустило издательство "Penguin Books", а именно 42 книги.

<a id='6-bullet'></a>
### Какой автор имеет самую высокую среднюю оценку среди книг толще 50 страниц?
[К оглавлению](#101-bullet)

In [7]:
#
query = '''
select
authors.author,
count(distinct ratings.rating_id),
avg(ratings.rating)
from
ratings
left join books on books.book_id = ratings.book_id
left join authors on authors.author_id = books.author_id
where books.num_pages > 50
group by authors.author
having count(distinct ratings.rating_id) > 50
order by avg desc
limit 10

'''

df = pd.io.sql.read_sql(query, con = engine)
df

,author,count,avg
0,J.K. Rowling/Mary GrandPré,312,4.288462
1,Agatha Christie,53,4.283019
2,Markus Zusak/Cao Xuân Việt Khương,53,4.264151
3,J.R.R. Tolkien,166,4.240964
4,Roald Dahl/Quentin Blake,62,4.209677
5,Louisa May Alcott,54,4.203704
6,Rick Riordan,84,4.130952
7,Arthur Golden,56,4.107143
8,Stephen King,106,4.009434
9,John Grisham,70,3.971429


Для уверенности в обоснованности рейтинга исключим их выборки авторов, на книги которых было высталено менее 50 оценок. Тогда самую высокую среднюю оценку среди книг толще 50 страниц имеет Дж. К. Роулинг (J.K. Rowling/Mary GrandPré), средний рейтинг её книг 4,28.

<a id='7-bullet'></a>
### Сколько в среднем обзоров было от пользователей, которые поставили больше 50 оценок?
[К оглавлению](#101-bullet)

In [8]:
#
query = '''
select
avg(cnt_rewiew) as avg_cnt_rewiew

from
        (select
        reviews.username,
        count(distinct review_id) as cnt_rewiew

        from
        reviews

        inner join (select
                    username,
                    count(distinct rating_id) as cnt_rating
                    from
                    ratings
                    group by username
                    having count(rating_id) > 50) as rat on rat.username = reviews.username

        group by reviews.username) as tt
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,avg_cnt_rewiew
0,24.333333


В среднем от пользоватлеей, которые поставили более 50 оценок было по 24 обзора.